In [ ]:
# ============================================
# 03_fx_agent.ipynb - Señal de cobertura FX (CORREGIDO)
# ============================================

!pip -q install pandas pyarrow

import os
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive', force_remount=False)

BASE = "/content/drive/MyDrive/investment_ai"
DIRS = {
    "reports": f"{BASE}/reports",
    "clean": f"{BASE}/data/clean"
}

# --- 1. Leer cartera enriquecida (fuente fiable de divisa_base) ---
ENRICHED_PATH = f"{DIRS['reports']}/portfolio_enriched_final.csv"
if not os.path.exists(ENRICHED_PATH):
    raise FileNotFoundError("Ejecuta primero 02_portfolio_exposure.ipynb")

pf = pd.read_csv(ENRICHED_PATH)
total_portfolio = pf["importe_actual_eur"].sum()

# Calcular exposición por divisa_base
exp_div = pf.groupby("divisa_base")["importe_actual_eur"].sum().to_frame()
exp_div.columns = ["importe_eur"]
exp_div.index.name = "divisa"

print("=== 🌍 EXPOSICIÓN POR DIVISA (desde cartera enriquecida) ===")
display(exp_div)

# --- 2. Leer tipos de cambio ---
fx_path = f"{DIRS['clean']}/fx_rates_clean.parquet"
if not os.path.exists(fx_path):
    raise FileNotFoundError("Ejecuta primero 01_data_prep.ipynb")

fx_prices = pd.read_parquet(fx_path)

# --- 3. Leer régimen de mercado ---
daily_path = f"{DIRS['reports']}/portfolio_daily_value.csv"
current_dd = 0.0
if os.path.exists(daily_path):
    daily = pd.read_csv(daily_path, index_col=0, parse_dates=True)
    current_dd = daily["drawdown"].min()

# --- 4. Configuración ---
FX_THRESHOLD = 0.10  # 10% del patrimonio
VOL_WINDOW = 30
TREND_WINDOW = 90

# --- 5. Generar señales ---
fx_signals = []

for divisa, row in exp_div.iterrows():
    if divisa == "EUR":
        continue

    exposure_eur = row["importe_eur"]
    exposure_pct = exposure_eur / total_portfolio

    print(f"\n🔍 Analizando divisa: {divisa} ({exposure_pct:.1%})")

    # Mapear divisa a par FX
    if divisa == "USD":
        fx_ticker = "EURUSD=X"
    elif divisa == "GBP":
        fx_ticker = "EURGBP=X"
    else:
        print(f"  ⚠️ Divisa no soportada: {divisa}")
        continue

    if fx_ticker not in fx_prices.columns:
        print(f"  ⚠️ Par FX no encontrado: {fx_ticker}")
        continue

    # Calcular tendencia y volatilidad
    fx_series = fx_prices[fx_ticker].dropna()
    if len(fx_series) < TREND_WINDOW:
        print(f"  ⚠️ Datos insuficientes para {fx_ticker}")
        continue

    # Tendencia: SMA 50 vs SMA 200
    sma_short = fx_series.rolling(50).mean().iloc[-1]
    sma_long = fx_series.rolling(200).mean().iloc[-1]
    trend_bearish = sma_short < sma_long  # EUR se debilita → cobertura

    # Volatilidad
    returns = fx_series.pct_change().dropna()
    vol_30d = returns.tail(VOL_WINDOW).std() * np.sqrt(252)
    high_vol = vol_30d > 0.10  # 10% anualizada

    # Régimen de riesgo
    risk_off = current_dd < -0.05  # Drawdown > 5%

    # Decisión
    hedge = False
    hedge_pct = 0

    if trend_bearish:
        hedge = True
        hedge_pct = 75
    if high_vol:
        hedge = True
        hedge_pct = max(hedge_pct, 50)
    if risk_off:
        hedge = True
        hedge_pct = 100

    print(f"  ✅ Tendencia bajista: {trend_bearish}, Alta vol: {high_vol}, Risk-off: {risk_off}")
    print(f"  📌 Cobertura: {'Sí' if hedge else 'No'} ({hedge_pct}%)")

    fx_signals.append({
        "divisa": divisa,
        "exposicion_eur": exposure_eur,
        "exposicion_%": exposure_pct,
        "cobertura_recomendada": "Sí" if hedge else "No",
        "%_cobertura": hedge_pct,
        "tendencia_bajista": trend_bearish,
        "alta_volatilidad": high_vol,
        "risk_off": risk_off,
        "volatilidad_anualizada_%": vol_30d,
        "sma_50": sma_short,
        "sma_200": sma_long
    })

# --- 6. Mostrar resultado ---
if fx_signals:
    signals_df = pd.DataFrame(fx_signals)
    print("\n=== 🌐 SEÑAL DE COBERTURA FX (DETALLADA) ===")
    display(signals_df.round(4))

    # Guardar
    signals_df.to_csv(f"{DIRS['reports']}/fx_hedge_signal.csv", index=False)
    print(f"\n✅ Señal guardada en: {DIRS['reports']}/fx_hedge_signal.csv")
else:
    print("\n✅ No se requiere cobertura FX en este momento.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
=== 🌍 EXPOSICIÓN POR DIVISA (desde cartera enriquecida) ===


,importe_eur
divisa,
EUR,639691.9
GBP,14191.0
USD,230141.4



🔍 Analizando divisa: GBP (1.6%)
  ✅ Tendencia bajista: False, Alta vol: False, Risk-off: False
  📌 Cobertura: No (0%)

🔍 Analizando divisa: USD (26.0%)
  ✅ Tendencia bajista: False, Alta vol: False, Risk-off: False
  📌 Cobertura: No (0%)

=== 🌐 SEÑAL DE COBERTURA FX (DETALLADA) ===


,divisa,exposicion_eur,exposicion_%,cobertura_recomendada,%_cobertura,tendencia_bajista,alta_volatilidad,risk_off,volatilidad_anualizada_%,sma_50,sma_200
0,GBP,14191.0,0.0161,No,0,False,False,False,0.0334,0.8671,0.8491
1,USD,230141.4,0.2603,No,0,False,False,False,0.0689,1.1682,1.1143



✅ Señal guardada en: /content/drive/MyDrive/investment_ai/reports/fx_hedge_signal.csv


In [2]:
# ============================================
# 03_fx_agent.ipynb - Señal de cobertura FX (MEJORADO + CONTRATO)
# ============================================

import os
import pandas as pd
import numpy as np
import json
from google.colab import drive

# Instalación segura
try:
    import pandas as pd
except ImportError:
    !pip -q install pandas pyarrow
    import pandas as pd

drive.mount('/content/drive', force_remount=False)

BASE = "/content/drive/MyDrive/investment_ai"
DIRS = {
    "reports": f"{BASE}/reports",
    "clean": f"{BASE}/data/clean"
}

# --- 1. Leer cartera enriquecida ---
ENRICHED_PATH = f"{DIRS['reports']}/portfolio_enriched_final.csv"
if not os.path.exists(ENRICHED_PATH):
    raise FileNotFoundError("❌ Ejecuta primero 02_portfolio_exposure.ipynb")

pf = pd.read_csv(ENRICHED_PATH)
total_portfolio = pf["importe_actual_eur"].sum()

if total_portfolio == 0:
    raise ValueError("❌ El valor total de la cartera es 0.")

# Calcular exposición por divisa_base
exp_div = pf.groupby("divisa_base")["importe_actual_eur"].sum().to_frame()
exp_div.columns = ["importe_eur"]
exp_div.index.name = "divisa"

print("=== 🌍 EXPOSICIÓN POR DIVISA (desde cartera enriquecida) ===")
print(exp_div.to_string())

# --- 2. Leer tipos de cambio ---
fx_path = f"{DIRS['clean']}/fx_rates_clean.parquet"
if not os.path.exists(fx_path):
    raise FileNotFoundError("❌ Ejecuta primero 01_data_prep.ipynb para generar fx_rates_clean.parquet")

fx_prices = pd.read_parquet(fx_path)
if fx_prices.empty:
    raise ValueError("❌ El archivo de tipos de cambio está vacío.")

# --- 3. Leer drawdown (de otro agente futuro) ---
daily_path = f"{DIRS['reports']}/portfolio_daily_value.csv"
current_dd = 0.0
risk_off = False
if os.path.exists(daily_path):
    try:
        daily = pd.read_csv(daily_path, index_col=0, parse_dates=True)
        if "drawdown" in daily.columns and not daily["drawdown"].empty:
            current_dd = daily["drawdown"].min()
            risk_off = current_dd < -0.05  # Drawdown > 5%
        print(f"📉 Drawdown actual: {current_dd:.2%} → Risk-off: {risk_off}")
    except Exception as e:
        print(f"⚠️ Error al leer drawdown: {e}. Asumiendo risk-off = False.")
else:
    print("⚠️ portfolio_daily_value.csv no encontrado. Asumiendo risk-off = False.")

# --- 4. Configuración ---
FX_THRESHOLD = 0.10  # 10% del patrimonio
VOL_WINDOW = 30
TREND_WINDOW = 90

# Mapeo extensible de divisas → pares FX
FX_MAP = {
    "USD": "EURUSD=X",
    "GBP": "EURGBP=X",
    "CHF": "EURCHF=X",
    "JPY": "EURJPY=X"
}

# --- 5. Generar señales ---
fx_signals = []

for divisa, row in exp_div.iterrows():
    if divisa == "EUR":
        continue

    exposure_eur = row["importe_eur"]
    exposure_pct = exposure_eur / total_portfolio

    if exposure_pct < FX_THRESHOLD:
        print(f"\n🔍 Ignorando {divisa}: exposición baja ({exposure_pct:.1%})")
        continue

    print(f"\n🔍 Analizando divisa: {divisa} ({exposure_pct:.1%})")

    # Mapear divisa a par FX
    if divisa not in FX_MAP:
        print(f"  ⚠️ Divisa no soportada: {divisa}. Soportadas: {list(FX_MAP.keys())}")
        continue

    fx_ticker = FX_MAP[divisa]

    if fx_ticker not in fx_prices.columns:
        print(f"  ⚠️ Par FX no encontrado: {fx_ticker}")
        continue

    # Calcular tendencia y volatilidad
    fx_series = fx_prices[fx_ticker].dropna()
    if len(fx_series) < max(200, TREND_WINDOW):
        print(f"  ⚠️ Datos insuficientes para {fx_ticker} (necesario: 200, actual: {len(fx_series)})")
        continue

    # Tendencia: SMA 50 vs SMA 200
    sma_short = fx_series.rolling(50).mean().iloc[-1]
    sma_long = fx_series.rolling(200).mean().iloc[-1]
    trend_bearish = sma_short < sma_long  # EUR se debilita → cobertura

    # Volatilidad
    returns = fx_series.pct_change().dropna()
    vol_30d = returns.tail(VOL_WINDOW).std() * np.sqrt(252)
    high_vol = vol_30d > 0.10  # 10% anualizada

    # Decisión
    hedge = False
    hedge_pct = 0

    if trend_bearish:
        hedge = True
        hedge_pct = 75
    if high_vol:
        hedge = True
        hedge_pct = max(hedge_pct, 50)
    if risk_off:
        hedge = True
        hedge_pct = 100

    print(f"  ✅ Tendencia bajista: {trend_bearish}, Alta vol: {high_vol}, Risk-off: {risk_off}")
    print(f"  📌 Cobertura: {'Sí' if hedge else 'No'} ({hedge_pct}%)")

    fx_signals.append({
        "divisa": divisa,
        "exposicion_eur": exposure_eur,
        "exposicion_%": exposure_pct,
        "cobertura_recomendada": "Sí" if hedge else "No",
        "%_cobertura": hedge_pct,
        "tendencia_bajista": trend_bearish,
        "alta_volatilidad": high_vol,
        "risk_off": risk_off,
        "volatilidad_anualizada_%": vol_30d,
        "sma_50": sma_short,
        "sma_200": sma_long
    })

# --- 6. Mostrar resultado y guardar ---
if fx_signals:
    signals_df = pd.DataFrame(fx_signals)
    print("\n=== 🌐 SEÑAL DE COBERTURA FX (DETALLADA) ===")
    print(signals_df.round(4).to_string())

    # Guardar CSV
    csv_path = f"{DIRS['reports']}/fx_hedge_signal.csv"
    signals_df.to_csv(csv_path, index=False)
    print(f"\n✅ Señal guardada en: {csv_path}")

    # Guardar JSON para orquestador
    divisas_a_cubrir = [s["divisa"] for s in fx_signals if s["cobertura_recomendada"] == "Sí"]
    recomendaciones = {
        s["divisa"]: {
            "exposicion_%": s["exposicion_%"],
            "%_cobertura": s["%_cobertura"],
            "razones": [r for r, v in [("Tendencia", s["tendencia_bajista"]), ("Volatilidad", s["alta_volatilidad"]), ("Risk-off", s["risk_off"])] if v]
        }
        for s in fx_signals if s["cobertura_recomendada"] == "Sí"
    }

    fx_summary = {
        "fecha": pd.Timestamp.now().strftime("%Y-%m-%d"),
        "divisas_a_cubrir": divisas_a_cubrir,
        "recomendaciones": recomendaciones,
        "total_divisas_analizadas": len(fx_signals)
    }

    json_path = f"{DIRS['reports']}/fx_hedge_latest.json"
    with open(json_path, "w") as f:
        json.dump(fx_summary, f, indent=2)
    print(f"✅ Resumen para orquestador: {json_path}")

else:
    print("\n✅ No se requiere cobertura FX en este momento.")
    # Guardar JSON vacío pero válido
    fx_summary = {
        "fecha": pd.Timestamp.now().strftime("%Y-%m-%d"),
        "divisas_a_cubrir": [],
        "recomendaciones": {},
        "total_divisas_analizadas": 0
    }
    json_path = f"{DIRS['reports']}/fx_hedge_latest.json"
    with open(json_path, "w") as f:
        json.dump(fx_summary, f, indent=2)
    print(f"✅ Resumen vacío guardado: {json_path}")

Mounted at /content/drive
=== 🌍 EXPOSICIÓN POR DIVISA (desde cartera enriquecida) ===
        importe_eur
divisa             
EUR        639691.9
GBP         14191.0
USD        230141.4
📉 Drawdown actual: -3.79% → Risk-off: False

🔍 Ignorando GBP: exposición baja (1.6%)

🔍 Analizando divisa: USD (26.0%)
  ✅ Tendencia bajista: False, Alta vol: False, Risk-off: False
  📌 Cobertura: No (0%)

=== 🌐 SEÑAL DE COBERTURA FX (DETALLADA) ===
  divisa  exposicion_eur  exposicion_% cobertura_recomendada  %_cobertura  tendencia_bajista  alta_volatilidad  risk_off  volatilidad_anualizada_%  sma_50  sma_200
0    USD        230141.4        0.2603                    No            0              False             False     False                    0.0689  1.1682   1.1143

✅ Señal guardada en: /content/drive/MyDrive/investment_ai/reports/fx_hedge_signal.csv
✅ Resumen para orquestador: /content/drive/MyDrive/investment_ai/reports/fx_hedge_latest.json
